In [ ]:
from itertools import combinations
import pathlib
import numpy as np
import numpy.typing as npt
import scipy.stats
import scipy.signal
import pandas as pd
from powltools.io.file import POwlFile
from powltools.analysis.recording import Recording
from powltools.analysis.recording import stim_level
from powltools.analysis.recording import stim_delay
from powltools.analysis.recording import stim_len
from powltools.analysis.recording import StimulusParams
from powltools.analysis.recording import group_by_param
from powltools.filters.offlinefilters import bandpass_filter

from xcorr_tools import binary_spiketrain
from xcorr_tools import make_psth_bins
from xcorr_tools import stim_psth_bins
from xcorr_tools import base_psth_bins
from xcorr_tools import trial_psth_bins
from xcorr_tools import cross_correlation
from xcorr_tools import get_peak
from xcorr_tools import fixed_var_stimuli
from xcorr_tools import iter_session_values
from xcorr_tools import filter_rate_level_flat
from xcorr_tools import filter_relative_level_flat
from xcorr_tools import get_power
from xcorr_tools import get_phaselocking
import glob


from matplotlib import pyplot as plt

In [ ]:
def get_latency(trace,stim_onset, time_bins):
    stim_start_index = np.searchsorted(time_bins, stim_onset)
    max_val = np.max(trace)
    max_ind = np.argmax(trace)
    half_ind = np.argmax(trace[stim_start_index:max_ind] > max_val/2)
    first_spike = time_bins[half_ind+stim_start_index]
    peak_time = time_bins[max_ind]
    return first_spike, peak_time

In [ ]:
# file_list = [
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231211_256_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231215_256_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231221_256_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231224_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231226_40_awake\03_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231228_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231230_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240108_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240312_256_awake\03_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240314_256_awake\01_rate_level.h5",    
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240315_256_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240325_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240329_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240402_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240403_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240420_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240423_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240425_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240426_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240429_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240430_54_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240505_53_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240508_53_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240509_53_awake\02_rate_level.h5"
#         ]

# file_list_out = [
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231211_256_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231215_256_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231221_256_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231224_40_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231226_40_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231228_40_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231230_40_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240108_40_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240312_256_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240314_256_awake\04_rate_level_out.h5",    
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240315_256_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240325_40_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240329_40_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240402_40_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240403_40_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240420_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240423_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240425_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240426_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240429_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240430_54_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240505_53_awake\04_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240508_53_awake\03_rate_level_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240509_53_awake\04_rate_level_out.h5"        
#         ]



# file_list = [
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231216_256_awake\04_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231222_256_awake\03_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231225_40_awake\03_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231227_40_awake\04_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231227_256_awake\04_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231229_40_awake\04_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231230_256_awake\03_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231231_40_awake\04_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240313_256_awake\05_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240316_256_awake\07_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240326_40_awake\01_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240330_40_awake\02_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240421_54_awake\02_rate_level_am_55.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240506_53_awake\02_rate_level_am_55.h5"
#         ]

# file_list_out = [
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231216_256_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231222_256_awake\05_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231225_40_awake\05_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231227_40_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231227_256_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231229_40_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231230_256_awake\05_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231231_40_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240313_256_awake\07_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240316_256_awake\05_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240326_40_awake\05_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240330_40_awake\06_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240421_54_awake\04_rate_level_am_55_out.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240506_53_awake\04_rate_level_am_55_out.h5"
#         ]

## Contra Competition
file_list = [
        r"C:\Users\andrea\Documents\Data\DualRegion\20240314_256_awake\01_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240315_256_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240325_40_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240329_40_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240402_40_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240403_40_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240420_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240423_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240425_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240426_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240429_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240430_54_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240505_53_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240508_53_awake\02_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240509_53_awake\02_rate_level.h5",
        ]

file_list_out = [
        r"C:\Users\andrea\Documents\Data\DualRegion\20240314_256_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240315_256_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240325_40_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240329_40_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240402_40_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240403_40_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240420_54_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240423_54_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240425_54_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240426_54_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240429_54_awake\05_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240430_54_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240505_53_awake\05_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240508_53_awake\06_rate_level_contra.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240509_53_awake\06_rate_level_contra.h5",
        ]


# file_list_am_55 = [
#     r"C:\Users\andrea\Documents\Data\DualRegion\20231225_40_awake\03_rate_level_am_55.h5",
    
# ]

#file_list = [r"C:\Users\andrea\Documents\Data\DualRegion\20240513_53_awake\02_rate_level.h5"]
#file_list_out = [r"C:\Users\andrea\Documents\Data\DualRegion\20240513_53_awake\04_rate_level_out.h5"]

In [ ]:
def get_psth(file_path):
    pf_single = POwlFile(file_path)
    rec = Recording(pf_single)
    date = rec.global_parameters()["session"]["date"]
    owl = rec.global_parameters()["session"]["owl"]
    hemisphere = rec.global_parameters()["session"]["hemisphere"]
    regions = rec.global_parameters()["regions"]
    channel_numbers = rec.channel_numbers()

    delays = rec.aggregate_stim_params(stim_delay)
    stimlens = rec.aggregate_stim_params(stim_len)
    psth_bins = [
        make_psth_bins(trialdelay, trialstim, binsize=0.001, offset=0)
        for trialdelay, trialstim in zip(delays, stimlens)
    ]
    base_bins = [
        make_psth_bins(trialdelay, trialstim, binsize=0.001, offset=0) - trialstim
        for trialdelay, trialstim in zip(delays, stimlens)
    ]
    # psth_bins = rec.aggregate_stim_params(stim_psth_bins)
    # base_bins = rec.aggregate_stim_params(base_psth_bins)
    trial_bins = [
        make_psth_bins(0, 2.5, binsize=0.001, offset=0)
        for trialdelay, trialstim in zip(delays, stimlens)
    ]

    varying_levels = np.array(rec.aggregate_stim_params(stim_level, stimulus_index=0))
    unique_levels = np.unique(varying_levels)
    fixed_azimuths = np.array(rec.aggregate_stim_params(lambda params: params["azi"]))
    fixed_elevations = np.array(rec.aggregate_stim_params(lambda params: params["ele"]))

    # Binary spiketrains during and before stimuli for all channels
    trial_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(binary_spiketrain, trial_bins, channel_number=chan)
        )
        for chan in list(channel_numbers)
    }

    stim_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(binary_spiketrain, psth_bins, channel_number=chan)
        )
        for chan in list(channel_numbers)
    }

    base_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(binary_spiketrain, base_bins, channel_number=chan)
        )
        for chan in list(channel_numbers)
    }

    df = []
    for chan in channel_numbers:
        psth_u1 = trial_spiketrains[chan]
        mean_psth = np.mean(psth_u1, axis=0)

        stim_psth_resp = stim_spiketrains[chan]
        base_psth_resp = base_spiketrains[chan]

        mean_stim_resp = np.sum([np.sum(trial) for trial in stim_psth_resp]) / len(
            stim_psth_resp
        )
        mean_base_resp = np.sum([np.sum(trial) for trial in base_psth_resp]) / len(
            base_psth_resp
        )
        std_base_resp = np.std([np.sum(trial) for trial in base_psth_resp])

        tmp = {
            "date": date,
            "owl": owl,
            "channel": chan,
            "azi": fixed_azimuths[0],
            "ele": fixed_elevations[0],
            "region": regions[str(chan)],
            "stim_delay": delays[0],
            "stim_duration": stimlens[0],
            "stim_resp": mean_stim_resp,
            "baseline_resp": mean_base_resp,
            "baseline_std": std_base_resp,
            "psth": mean_psth,
            "hemisphere": hemisphere,
        }
        df.append(tmp)
    df = pd.DataFrame(df)
    return df


def psth_properties(psth, stim_delay, stim_len):

    time_bins = np.arange(0, len(psth)) * 0.001
    time_bins = np.array([round(val,3) for val in time_bins])
    
    max_stim_resp = np.max(
        psth[(time_bins > stim_delay) & (time_bins <= stim_delay + 0.1)]
    )
    
    base_psth = np.mean(psth[(time_bins>= 0) & (time_bins< stim_len)])
    base_psth_std = np.std(psth[(time_bins>= 0) & (time_bins< stim_len)])
    

    properties = {
        "max_stim": max_stim_resp,
        "base_psth": base_psth,
        "base_psth_std": base_psth_std
    }

    return properties

In [ ]:
auditory_units_df = []
ot_psth = []
fore_psth = []
for file, fileout in zip(file_list[:], file_list_out[:]):
    driver_position = get_psth(file)
    competitor_position = get_psth(fileout)

    driver_position["stim_position"] = ["driver"] * len(driver_position)
    competitor_position["stim_position"] = ["competitor"] * len(competitor_position)

    driver_position = driver_position.set_index(["date", "owl", "channel"])
    competitor_position = competitor_position.set_index(["date", "owl", "channel"])

    merge = driver_position.join(
        competitor_position, how="inner", rsuffix="_competitor"
    )
    
    if len(merge) == 0:
        print(file, fileout)
    
    for index in np.unique(merge.index):
        channel_df = merge.loc[index]
        driver_psth = channel_df["psth"]
        delay = channel_df["stim_delay"]
        stimlen = channel_df["stim_duration"]


        if channel_df["region"] == "OT":
            properties = psth_properties(driver_psth, delay, stimlen)
            if (
                channel_df["stim_resp"]
                > channel_df["baseline_resp"] + 2 * channel_df["baseline_std"]
            ) and (
                channel_df['stim_resp_competitor'] <= channel_df["baseline_resp_competitor"] + 2* channel_df['baseline_std']
            ):
                try:
                    time_bins = np.arange(0, len(driver_psth)) * 0.001
                    time_bins = np.array([round(val,3) for val in time_bins])
                    time_firstspike, time_peak = get_latency(driver_psth, delay, time_bins)

                    tmp = {
                        "date": index[0],
                        "owl": index[1],
                        "hemisphere": channel_df["hemisphere"],
                        "channel": index[2],
                        "azi": channel_df["azi"],
                        "ele": channel_df["ele"],
                        "azi_competitor": channel_df["azi_competitor"],
                        "ele_competitor": channel_df["ele_competitor"],
                        "region": channel_df["region"],
                        "stimulus_resp": channel_df["stim_resp"],
                        "base_resp": channel_df["baseline_resp"],
                        "base_std": channel_df["baseline_std"],
                        "maxresp_latency": time_peak,
                        "firstspk_latency": time_firstspike,
                        "psth_driver": driver_psth,
                        "psth_competitor": channel_df["psth_competitor"],
                    }
                    auditory_units_df.append(tmp)
#                     plt.figure()
#                     plt.title(f'channel {index[2]}')
#                     plt.plot(driver_psth, color = 'k')
#                     plt.plot(channel_df["psth_competitor"], color = 'r')
                except:
                    pass


                
            elif (
                properties["max_stim"]
                > properties["base_psth"]
                + 9 * properties["base_psth_std"]
            ) & (properties['max_stim'] > 0.10):
                try:
                    time_bins = np.arange(0, len(driver_psth)) * 0.001
                    time_bins = np.array([round(val,3) for val in time_bins])
                    time_firstspike, time_peak = get_latency(driver_psth, delay, time_bins)
                    tmp = {
                        "date": index[0],
                        "owl": index[1],
                        "hemisphere": channel_df["hemisphere"],
                        "channel": index[2],
                        "azi": channel_df["azi"],
                        "ele": channel_df["ele"],
                        "azi_competitor": channel_df["azi_competitor"],
                        "ele_competitor": channel_df["ele_competitor"],
                        "region": channel_df["region"],
                        "stimulus_resp": channel_df["stim_resp"],
                        "base_resp": channel_df["baseline_resp"],
                        "base_std": channel_df["baseline_std"],
                        "maxresp_latency": time_peak,
                        "firstspk_latency": time_firstspike,
                        "psth_driver": driver_psth,
                        "psth_competitor": channel_df["psth_competitor"],
                    }
                    auditory_units_df.append(tmp)
#                     plt.figure()
#                     plt.title(f'channel {index[2]}')
#                     plt.plot(driver_psth, color = 'k')
#                     plt.plot(channel_df["psth_competitor"], color = 'r')
                except:
                    pass


        elif channel_df["region"] == "Forebrain":
            
            if channel_df["stim_resp"] >= channel_df["stim_resp_competitor"]:
                psth = driver_psth
                properties = psth_properties(psth, delay, stimlen)
            elif channel_df["stim_resp"] < channel_df["stim_resp_competitor"]:
                psth = channel_df['psth_competitor']
                properties = psth_properties(
                    psth, delay, stimlen
                )

            if (
                channel_df["stim_resp"]
                > channel_df["baseline_resp"] + 2 * channel_df["baseline_std"]
            ) or (
                channel_df["stim_resp_competitor"]
                > channel_df["baseline_resp_competitor"]
                + 2 * channel_df["baseline_std_competitor"]
            ):
                try:
                    time_bins = np.arange(0, len(psth)) * 0.001
                    time_bins = np.array([round(val,3) for val in time_bins])
                    time_firstspike, time_peak = get_latency(psth, delay, time_bins)
                    tmp = {
                        "date": index[0],
                        "owl": index[1],
                        "hemisphere": channel_df["hemisphere"],
                        "channel": index[2],
                        "azi": channel_df["azi"],
                        "ele": channel_df["ele"],
                        "azi_competitor": channel_df["azi_competitor"],
                        "ele_competitor": channel_df["ele_competitor"],
                        "region": channel_df["region"],
                        "stimulus_resp": channel_df["stim_resp"],
                        "base_resp": channel_df["baseline_resp"],
                        "base_std": channel_df["baseline_std"],
                        "maxresp_latency": time_peak,
                        "firstspk_latency": time_firstspike,
                        "psth_driver": driver_psth,
                        "psth_competitor": channel_df["psth_competitor"],
                    }
                    auditory_units_df.append(tmp)
#                     plt.figure()
#                     plt.title(f'channel {index[2]}')
#                     plt.plot(mean_psth, color = 'k')
#                     plt.plot(channel_df["psth_competitor"], color = 'r')

                except:
                    pass


            elif (
                properties["max_stim"]
                > properties["base_psth"]
                + 9 * properties["base_psth_std"]
            ) & (properties['max_stim'] > 0.10):
                try:
                    time_bins = np.arange(0, len(psth)) * 0.001
                    time_bins = np.array([round(val,3) for val in time_bins])
                    time_firstspike, time_peak = get_latency(psth, delay, time_bins)
                    tmp = {
                        "date": index[0],
                        "owl": index[1],
                        "hemisphere": channel_df["hemisphere"],
                        "channel": index[2],
                        "azi": channel_df["azi"],
                        "ele": channel_df["ele"],
                        "azi_competitor": channel_df["azi_competitor"],
                        "ele_competitor": channel_df["ele_competitor"],
                        "region": channel_df["region"],
                        "stimulus_resp": channel_df["stim_resp"],
                        "base_resp": channel_df["baseline_resp"],
                        "base_std": channel_df["baseline_std"],
                        "maxresp_latency": time_peak,
                        "firstspk_latency": time_firstspike,
                        "psth_driver": driver_psth,
                        "psth_competitor": channel_df["psth_competitor"],
                    }
                    auditory_units_df.append(tmp)
#                     plt.figure()
#                     plt.title(f'channel {index[2]}')
#                     plt.plot(mean_psth, color = 'k')
#                     plt.plot(channel_df["psth_competitor"], color = 'r')

                except:
                    pass
                

auditory_units_df = pd.DataFrame(auditory_units_df)

In [ ]:
OT_units = auditory_units_df[auditory_units_df['region'] == 'OT']['psth_driver']
Forebrain_units = auditory_units_df[auditory_units_df['region'] == 'Forebrain']['psth_driver']

plt.figure()
plt.plot(time_bins, np.mean(OT_units, axis =0), color = 'b', label = 'OT')
plt.plot(time_bins, np.mean(Forebrain_units, axis =0), color = 'r', label = 'Forebrain')
#plt.xlim(.50,1.25)
plt.legend()
plt.xlim(0.9,1.2)

print(len(OT_units))
print(len(Forebrain_units))

In [ ]:
plt.figure()
plt.hist(auditory_units_df[auditory_units_df['region']=='OT']['maxresp_latency'], label = 'OT')
plt.hist(auditory_units_df[(auditory_units_df['region']=='Forebrain') ]['maxresp_latency'], label = 'Fore')
#plt.hist(auditory_units_df[(auditory_units_df['region']=='Forebrain') & (auditory_units_df['owl']== '256')]['firstspk_latency'], label = 'Fore')
#plt.xlim(1.0,1.10)
plt.legend()

In [ ]:
auditory_units_df[auditory_units_df['region'] == 'Forebrain']
# auditory_units_df = auditory_units_df[
#     (auditory_units_df["channel"] != 1)
#     & (auditory_units_df["channel"] != 13)
# ]
# display(auditory_units_df)

In [ ]:
# date = auditory_units_df.iloc[0]['date']
# owl = auditory_units_df.iloc[0]['owl']

# auditory_units_df.to_csv(
#     fr"C:\Users\andrea\Documents\Python\analysis_with_powl\pooled_data_excel\DualRegion_auditory_units\auditory_units_{date}_{owl}.csv",
#     header=True,
#     index=False,
# )

auditory_units_df.to_csv(
    fr"C:\Users\andrea\Documents\Python\analysis_with_powl\pooled_data_excel\DualRegion_auditory_units_amnoise\auditory_units_contra_flatnoise.csv",
    header=True,
    index=False,
)

# Compile csv files for pooled data

In [ ]:
filenames = glob.glob(r"C:\Users\andrea\Documents\Python\analysis_with_powl\pooled_data_excel\DualRegion_auditory_units/*.csv")
pooled_data = []
for filename in filenames:
    df = pd.read_csv(fr'{filename}')
#    print(filename[43:51])
    pooled_data.append(df)
pooled_data = pd.concat(pooled_data)
display(pooled_data)

In [ ]:
pooled_data.to_csv(
    fr"C:\Users\andrea\Documents\Python\analysis_with_powl\pooled_data_excel\DualRegion_auditory_units\auditory_units_combined.csv",
    header=True,
    index=False,
)

In [ ]:
pooled_data[pooled_data['region'] == 'OT'][-5:]

# Latency by Jose's Suggestion

In [ ]:
paths = [r"C:\Users\andrea\Documents\Data\DualRegion\20240314_256_awake\01_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240312_256_awake\03_rate_level.h5",
        r"C:\Users\andrea\Documents\Data\DualRegion\20240315_256_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231224_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231226_40_awake\03_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231228_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20231230_40_awake\02_rate_level.h5",
#         r"C:\Users\andrea\Documents\Data\DualRegion\20240108_40_awake\02_rate_level.h5"
        ]



In [ ]:
def get_latency_2SD(mean_psth, threshold, time_bins, delay):
    stim_onset = np.where(time_bins == delay)[0][0]
    max_val = np.argmax(trace)    
    index = np.where(mean_psth[stim_onset:max_val] > threshold)[0][0]
    response_latency = time_bins[index + stim_onset]

#    max_ind = np.argmax(trace)
#    half_ind = np.argmax(trace[:max_ind] > max_val/2)
#    first_spike = time_bins[0][half_ind]
#    peak_time = time_bins[0][max_ind]
    return response_latency

In [ ]:
auditory_units_df = []
for file in paths:
    pf_single = POwlFile(file)
    rec = Recording(pf_single)
    hemisphere: str = rec.global_parameters()["session"]["hemisphere"]  # type: ignore
    channels = rec.global_parameters()['regions']
    #channel_numbers = rec.channel_numbers()
#    channels = {ch: "OT" for ch in rec.channel_numbers()}
    # channels = {"1": "ICx", 
    #             "2": "ICx",
    #             "3": "ICx",
    #             "4": "ICx",
    #             "5": 'ICx',
    #             "6": 'ICx'}

    delays = rec.aggregate_stim_params(stim_delay)
    stimlens = rec.aggregate_stim_params(stim_len)
    psth_bins = [make_psth_bins(trialdelay, trialstim, binsize =0.001, offset = 0) for trialdelay, trialstim in zip(delays, stimlens)]
    base_bins = [make_psth_bins(trialdelay, trialstim, binsize = 0.001, offset = 0) - trialstim for trialdelay, trialstim in zip(delays, stimlens)]
    trial_bins = [make_psth_bins(0 ,2.5, binsize = 0.001, offset = 0) for trialdelay, trialstim in zip(delays, stimlens)]


    varying_levels = np.array(
        rec.aggregate_stim_params(stim_level, stimulus_index=0)
    )
    unique_levels = np.unique(varying_levels)
    fixed_azimuths = np.array(
        rec.aggregate_stim_params(lambda params: params["azi"])
    )
    fixed_elevations = np.array(
        rec.aggregate_stim_params(lambda params: params["ele"])
    )

    # Binary spiketrains during and before stimuli for all channels
    trial_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(
                binary_spiketrain, trial_bins, channel_number=chan
            )
        )
        for chan in list(channels.keys())
    }

    stim_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(
                binary_spiketrain, psth_bins, channel_number=chan
            )
        )
        for chan in list(channels.keys())
    }    

    base_spiketrains: dict[int, npt.NDArray[np.int_]] = {
        chan: np.vstack(
            rec.aggregrate_spikes(
                binary_spiketrain, base_bins, channel_number=chan
            )
        )
        for chan in list(channels.keys())
    }    

    ot_psth = []
    fore_psth = []
    for chan, region in channels.items():
        
        psth_u1 = trial_spiketrains[chan]
        mean_psth = np.mean(psth_u1, axis = 0)

        max_stim_ind = np.argmax(mean_psth[(trial_bins[0][:-1] >= delays[0]) & (trial_bins[0][:-1] <= delays[0] + 0.05)])
        delay_ind = np.where(trial_bins[0] == delays[0])[0]
        max_latency = trial_bins[0][max_stim_ind] + delays[0]


        stim_psth_resp = stim_spiketrains[chan]
        base_psth_resp = base_spiketrains[chan]
        
        max_stim = np.max(mean_psth[(trial_bins[0][:-1] > delays[0]) & (trial_bins[0][:-1] <= delays[0] + 0.1)])
        base_psth = np.mean(mean_psth[(trial_bins[0][:-1] >  delays[0]-stimlens[0]) & (trial_bins[0][:-1] <= delays[0])])
        std_psth = np.std(mean_psth[(trial_bins[0][:-1] >  delays[0]-stimlens[0]) & (trial_bins[0][:-1] <= delays[0])])
        
                

        mean_stim_resp = np.sum([np.sum(trial) for trial in stim_psth_resp])/len(stim_psth_resp)
        mean_base_resp = np.sum([np.sum(trial) for trial in base_psth_resp])/len(base_psth_resp) 
        std_base_resp = np.std([np.sum(trial) for trial in base_psth_resp])
#         if int(chan) <= 16:
#             ot_psth.append(mean_psth)
#         else: 
#            fore_psth.append(mean_psth)
        if mean_stim_resp > mean_base_resp + 3*std_base_resp:

            mean_base_psth = np.mean(base_psth)
            base_std = np.std(base_psth)
            
            threshold = mean_base_psth + 5*std_psth
            latency = get_latency_2SD(mean_psth, threshold, trial_bins[0], delays[0])
            
#             plt.figure()
#             plt.plot(region)
#             plt.plot(trial_bins[0][:-1], mean_psth, color = 'b' if region == 'OT' else 'r')
#             plt.axvline(latency)
#             plt.axhline(threshold)
            
            tmp = {
                'date': rec.global_parameters()['session']['date'],
                'owl': rec.global_parameters()['session']['owl'],
                'hemisphere': rec.global_parameters()['session']['hemisphere'],
                'channel': chan,
                "azi": fixed_azimuths[0],
                "ele": fixed_elevations[0],
                'region': region,
                'stimulus_resp': mean_stim_resp,
                'base_resp': mean_base_resp,
                'base_std': std_base_resp,
                'response_latency': latency
            }
            auditory_units_df.append(tmp)
            if region == 'OT':
                ot_psth.append(mean_psth)
            else: 
                fore_psth.append(mean_psth)  
        elif max_stim > base_psth + 10*std_psth:
            mean_base_psth = np.mean(base_psth)
            base_std = np.std(base_psth)
            
            threshold = mean_base_psth + 2*std_psth
            latency = get_latency_2SD(mean_psth, threshold, trial_bins[0], delays[0])            
#             plt.figure()
#             plt.plot(region)
#             plt.plot(trial_bins[0][:-1], mean_psth, color = 'b' if region == 'OT' else 'r')
#             plt.axvline(latency)
#             plt.axhline(threshold)
            
            tmp = {
                'date': rec.global_parameters()['session']['date'],
                'owl': rec.global_parameters()['session']['owl'],
                'hemisphere': rec.global_parameters()['session']['hemisphere'],
                'channel': chan,
                "azi": fixed_azimuths[0],
                "ele": fixed_elevations[0],
                'region': region,
                'stimulus_resp': mean_stim_resp,
                'base_resp': mean_base_resp,
                'base_std': std_base_resp,
                'response_latency': latency
            }
            auditory_units_df.append(tmp)
            if region == 'OT':
                ot_psth.append(mean_psth)
            else: 
                fore_psth.append(mean_psth)

auditory_units_df = pd.DataFrame(auditory_units_df)
display(auditory_units_df)

In [ ]:
plt.figure()

plt.hist(auditory_units_df[auditory_units_df['region'] == 'OT']['response_latency'], label = 'OT')
plt.hist(auditory_units_df[auditory_units_df['region'] == 'Forebrain']['response_latency'], label = 'Forebrain')
plt.legend()

# Frequency Tuning

In [ ]:
def stim_frequency(stim_params: StimulusParams) -> int:
    return stim_params["stim_func_factory_kwargs"]["frequency"]

In [ ]:
pf = POwlFile(r"C:\Users\andrea\Documents\Data\DualRegion_anesthetized\20240501_53_anesthetized\22_frequency.h5")
rec = Recording(pf)
channels = rec.global_parameters()['regions']
stim_duration = set(rec.aggregate_stim_params(stim_len)).pop()

depth_order = [30, 27, 19, 22, 18, 23, 17, 24, 20, 21, 32, 25, 31, 26, 29, 28]

#OT
# depth_order = [14, 11, 3, 6, 2, 7, 1, 8, 4, 5, 16, 9, 15, 10, 13, 12]

#for chan, region in channels.items():
for chan in depth_order:
    if chan == 8:
        continue
    resp = rec.response_rates(channel_number=chan, stimulus_index=0)
    trial_frequencies = np.array(
        rec.aggregate_stim_params(stim_frequency, stimulus_index=0)
    )
    resp_by_frequency = group_by_param(resp, trial_frequencies)
    x_freq = list(resp_by_frequency.keys())
    y_vals = [np.mean(resp_by_frequency[key]) for key in resp_by_frequency.keys()]
    sem = [scipy.stats.sem(resp_by_frequency[key]) for key in resp_by_frequency.keys()]
    
    plt.figure()
    plt.title(f'Channel {chan}')
    plt.errorbar(resp_by_frequency.keys(), y_vals,sem, label = x_freq[np.argmax(y_vals)])
    plt.axvline(x_freq[np.argmax(y_vals)], color = 'k')
    plt.legend()

# Spatial Receptive Field and Latencies